# modelo 07 - xgboost

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [3]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

{'S-unknown', 'openspace', 'r_openspace', 'roof'}

In [4]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

{'D', 'S'}

In [5]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [6]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
294,IDsbMkn,[],[],[3],"[[(265.216, 193.87733333333333), (288.0, 211.6...","[1, 0, 0, 0, 1, 0]"
1845,IDiAp2V2hhCWn,[],[],[1],"[[(284.416, 176.46933333333334), (291.584, 213...","[1, 0, 0, 0, 1, 0]"
1000,IDHT1abkaVudn,[],[],[2],"[[(129.92, 290.816), (263.808, 321.70666666666...","[1, 0, 0, 0, 1, 0]"


---

In [7]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
import xgboost as xgb
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [8]:
# Load EfficientNetB0 without the top layer
effnet = EfficientNetB5(weights="imagenet", include_top=False, pooling="avg")

datagen = ImageDataGenerator(
    rotation_range=40,       # Random rotation range in degrees
    width_shift_range=0.2,   # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,         # Random shear transformation
    horizontal_flip=True,    # Random horizontal flip
    fill_mode='nearest',      # How to fill in newly created pixels (nearest pixel)
    brightness_range=[0.2, 1.8],
    
)

# Function to extract features from an image
def extract_image_features(image_path):

    image_path = "images/" + image_path + ".jpg"

    img = cv2.imread(image_path)
    img = cv2.resize(img, (456, 456))
    img = preprocess_input(img)  # Normalize image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    augmented_img = datagen.random_transform(img[0].copy())
    augmented_img = np.expand_dims(augmented_img, axis=0)  # Add batch dimension


    features = effnet.predict(augmented_img, verbose=0)  # Extract features
    return features.flatten()  # Convert to 1D

2025-03-20 23:26:56.733584: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-03-20 23:26:56.733614: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-20 23:26:56.733619: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-20 23:26:56.733635: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-20 23:26:56.733645: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
"""image_features = []
additional_features = []
targets = []
for i, row in df_train.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(df_train)}")

    image_features.append(extract_image_features(row["img_id"]))
    additional_features.append(row["more_features"])

    nr_boil = sum(row["nr_boil"])
    nr_pan = sum(row["nr_pan"])
    targets.append([nr_boil, nr_pan])

additional_features = np.array(additional_features)
targets = np.array(targets)

X_train = np.hstack((image_features, additional_features))
y_train = targets"""
''

''

In [10]:
"""image_features = []
additional_features = []
targets = []
for i, row in df_val.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(df_val)}")

    image_features.append(extract_image_features(row["img_id"]))
    additional_features.append(row["more_features"])

    nr_boil = sum(row["nr_boil"])
    nr_pan = sum(row["nr_pan"])
    targets.append([nr_boil, nr_pan])

additional_features = np.array(additional_features)
targets = np.array(targets)

X_val = np.hstack((image_features, additional_features))
y_val = targets"""
''

''

In [11]:
import pickle

"""a = {"X_train": X_train, "y_train": y_train, "X_val": X_val, "y_val": y_val}
with open('ModelBuild/model06_data.pkl', 'wb') as f:
    pickle.dump(a, f)"""

with open('ModelBuild/model06_data.pkl', 'rb') as f:
    a = pickle.load(f)

X_train = a["X_train"]
y_train = a["y_train"]
X_val = a["X_val"]
y_val = a["y_val"]

In [12]:
{'estimator__subsample': 0.5,
 'estimator__reg_lambda': 10,
 'estimator__reg_alpha': 5,
 'estimator__n_estimators': 500,
 'estimator__max_depth': 7,
 'estimator__learning_rate': 0.01,
 'estimator__gamma': 0.2,
 'estimator__colsample_bytree': 0.5}
 
 # public score: 3.000000 rounded

{'estimator__subsample': 0.5,
 'estimator__reg_lambda': 10,
 'estimator__reg_alpha': 5,
 'estimator__n_estimators': 500,
 'estimator__max_depth': 7,
 'estimator__learning_rate': 0.01,
 'estimator__gamma': 0.2,
 'estimator__colsample_bytree': 0.5}

In [ ]:
{'estimator__subsample': 0.5,
 'estimator__reg_lambda': 50,
 'estimator__reg_alpha': 5,
 'estimator__n_estimators': 300,
 'estimator__max_depth': 7,
 'estimator__learning_rate': 0.01,
 'estimator__gamma': 0,
 'estimator__colsample_bytree': 0.3}

# public score: 2.948509485 rounded

In [ ]:
xgb_regressor = MultiOutputRegressor(xgb.XGBRegressor(objective="reg:squarederror",
                                                      random_state=42))

param_dist = {
    "estimator__n_estimators": [100, 300, 500, 700],  # More trees improve results
    "estimator__learning_rate": [0.005, 0.01, 0.05],  # Lower LR prevents overfitting
    "estimator__max_depth": [3, 5, 7],  # Limit tree depth for large feature sets
    "estimator__subsample": [0.4, 0.5, 0.7, 0.9],  # Row sampling
    "estimator__colsample_bytree": [0.3, 0.5, 0.6],  # Feature sampling (important for 2K features)
    "estimator__reg_lambda": [5, 10, 20, 50],  # L2 Regularization
    "estimator__reg_alpha": [5, 10, 50],  # L1 Regularization
    "estimator__gamma": [0, 0.1, 0.2, 0.5],  # Controls tree split sensitivity
}

param_dist = {'estimator__subsample': [0.5],
 'estimator__reg_lambda': [50],
 'estimator__reg_alpha': [5],
 'estimator__n_estimators': [300],
 'estimator__max_depth': [7],
 'estimator__learning_rate': [0.01],
 'estimator__gamma': [0],
 'estimator__colsample_bytree': [0.3]}

# 5. Run RandomizedSearchCV
random_search = RandomizedSearchCV(xgb_regressor, param_distributions=param_dist, 
                                   n_iter=1, scoring="neg_mean_absolute_error",
                                   cv=7, verbose=1, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

Fitting 7 folds for each of 60 candidates, totalling 420 fits


/opt/anaconda3/envs/CAA01/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=7,
                   estimator=MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                                         booster=None,
                                                                         callbacks=None,
                                                                         colsample_bylevel=None,
                                                                         colsample_bynode=None,
                                                                         colsample_bytree=None,
                                                                         device=None,
                                                                         early_stopping_rounds=None,
                                                                         enable_categorical=False,
                                                                         eval_metric=None,
                                                                         feature_types=None,
                                                                         feature_weights=None,
                                                                         gamma=None,
                                                                         grow_policy=None,
                                                                         importance_t...
                   param_distributions={'estimator__colsample_bytree': [0.3,
                                                                        0.5,
                                                                        0.6],
                                        'estimator__gamma': [0, 0.1, 0.2, 0.5],
                                        'estimator__learning_rate': [0.005,
                                                                     0.01,
                                                                     0.05],
                                        'estimator__max_depth': [3, 5, 7],
                                        'estimator__n_estimators': [100, 300,
                                                                    500, 700],
                                        'estimator__reg_alpha': [5, 10, 50],
                                        'estimator__reg_lambda': [5, 10, 20,
                                                                  50],
                                        'estimator__subsample': [0.4, 0.5, 0.7,
                                                                 0.9]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=1)

In [14]:
best_params = random_search.best_params_
best_params

{'estimator__subsample': 0.5,
 'estimator__reg_lambda': 50,
 'estimator__reg_alpha': 5,
 'estimator__n_estimators': 300,
 'estimator__max_depth': 7,
 'estimator__learning_rate': 0.01,
 'estimator__gamma': 0,
 'estimator__colsample_bytree': 0.3}

In [15]:
best_model = random_search.best_estimator_

In [16]:
y_pred = best_model.predict(X_val)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_val, y_pred)
print(f"Test MAE: {mae}")

y_pred = best_model.predict(X_train)
y_pred = np.round(y_pred).astype(int)
mae = mean_absolute_error(y_train, y_pred)
print(f"Train MAE: {mae}")

Test MAE: 2.1811475409836065
Train MAE: 1.5391553915539156


---

In [19]:
submission = pd.read_csv("zindi_files/SampleSubmission.csv")
test = pd.read_csv("Test.csv")

In [20]:
test.head()

,ID,img_origin,placement
0,ID00qprY,D,roof
1,ID01AciUc,D,roof
2,ID0328D,D,roof
3,ID05WxObCFTs9,D,roof
4,ID06AdCmLMlkO,S,S-unknown


In [21]:
submission.head()

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,0
2,ID01AciUc_boil,0
3,ID01AciUc_pan,0
4,ID0328D_boil,0


In [22]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["placement"], inplace=True)

hot_encod_img_placement(test)

In [23]:
test.head()

,ID,more_features
0,ID00qprY,"[1, 0, 0, 0, 1, 0]"
1,ID01AciUc,"[1, 0, 0, 0, 1, 0]"
2,ID0328D,"[1, 0, 0, 0, 1, 0]"
3,ID05WxObCFTs9,"[1, 0, 0, 0, 1, 0]"
4,ID06AdCmLMlkO,"[0, 0, 0, 1, 0, 1]"


In [24]:
image_features = []
additional_features = []
targets = []
for i, row in test.iterrows():

    if i % 250 == 0:
        print(f"{i} of {len(test)}")

    image_features.append(extract_image_features(row["ID"]))
    additional_features.append(row["more_features"])


additional_features = np.array(additional_features)
targets = np.array(targets)

X_test = np.hstack((image_features, additional_features))

0 of 1107


2025-03-21 03:14:29.508333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250 of 1107
500 of 1107
750 of 1107
1000 of 1107


In [25]:
pred = best_model.predict(X_test)

pred

array([[8.7564997e-02, 1.6448802e+00],
       [4.3870565e-03, 2.5992115e+00],
       [1.3634238e-01, 4.0423617e+00],
       ...,
       [6.5784097e-01, 6.2314677e+00],
       [1.5520574e-02, 2.6234956e+00],
       [3.5987385e-02, 4.0052714e+00]], dtype=float32)

In [26]:
for i, row in test.iterrows():
    submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]
    submission.loc[submission["ID"] == f"{row['ID']}_pan", "Target"] = pred[i][1]

submission.head()

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_4694/1570848020.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.08756499737501144' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission.loc[submission["ID"] == f"{row['ID']}_boil", "Target"] = pred[i][0]


,ID,Target
0,ID00qprY_boil,0.087565
1,ID00qprY_pan,1.644880
2,ID01AciUc_boil,0.004387
3,ID01AciUc_pan,2.599211
4,ID0328D_boil,0.136342


In [27]:
submission.to_csv("zindi_files/model07_submission.csv", index=False)

In [28]:
submission_rounded = submission.copy()
submission_rounded["Target"] = submission_rounded["Target"].round().astype(int)
submission_rounded.to_csv("zindi_files/model07_submission_rounded.csv", index=False)

**public_score** 2.948509485